## Extract Multi-modal Emotion Features

### Config Setting

In [3]:
import clip
import torch
import os

device = "cuda" if torch.cuda.is_available() else "cpu"
# DATASET is the dataset name model trained on, e.g. ESD, RAF, MELD
DATASET = "ESD" # "RAF" "MELD"

PROJECT_PATH = os.path.join('/', *os.getcwd().split(os.sep)[:-2])
# PRETRAIN_MODEL is the pretrained model name, e.g. ViT-B/32
PRETRAIN_MODEL = "ViT-B/32"
# PRETRAIN_MODEL_PATH is the pretrained model path, e.g. EPAlign/ckpt/base
PRETRAIN_MODEL_PATH = f"{PROJECT_PATH}/EPAlign/ckpt/base"
# FINETUNE_MODEL is the finetuned model path, e.g. EPAlign/ckpt/ESD/best_model.pt
FINETUNE_MODEL = f"{PROJECT_PATH}/EPAlign/ckpt/{DATASET}/best_model.pt"
# EMO_FEATURE_SAVE_PATH is the path to save the extracted emotion features, e.g. EPAlign/mmefeature/ESD
EMO_FEATURE_SAVE_PATH = f"{PROJECT_PATH}/EPAlign/mmefeature/{DATASET}"
os.makedirs(EMO_FEATURE_SAVE_PATH, exist_ok=True)

### Load Model

In [4]:
model, preprocess = clip.load(PRETRAIN_MODEL, device=device, jit=False, download_root=PRETRAIN_MODEL_PATH)
model.load_state_dict(torch.load(FINETUNE_MODEL))

<All keys matched successfully>

### Prompt Setting

In [5]:
# labels = ["normal neutral","normal calm","normal happy","normal sad","normal angry","normal fearful","normal disgust","normal surprised","strong clam","strong happy","strong sad","strong angry","strong fearful","strong disgust","strong surprised"]
labels = ["neutral", "happy", "sad", "angry", "surprise"]


# prompt = [f'talk in {label} emotion', for label in labels]
prompt = [f'A person speaking with a feeling of {label}' for label in labels]

### Extract & Save mmefeature

In [7]:
emo_prompt = clip.tokenize(prompt).to(device)

with torch.no_grad():
    emo_prompt_features = model.encode_text(emo_prompt)
    emo_prompt_features /= emo_prompt_features.norm(dim=-1, keepdim=True)

emo_prompt_features = emo_prompt_features.split(1, dim=0)

for i in range(len(emo_prompt_features)):
    # torch.save(emo_prompt_features[i].squeeze(), f"{emo_feature_save_path}/{tag[i].split(' ')[2]}and{tag[i].split(' ')[4]}.pt")
    torch.save(emo_prompt_features[i].squeeze(), f"{EMO_FEATURE_SAVE_PATH}/{labels[i].split(' ')[-1]}.pt")


### Check Feature Shape

In [ ]:
import os
for file in os.listdir(EMO_FEATURE_SAVE_PATH):
    efeature = torch.load(f"{EMO_FEATURE_SAVE_PATH}/{file}", map_location=torch.device('cpu'))
    if efeature.shape[0] != 512:
        print(file)